In [ ]:
from PIL import Image

# Function to embed message in image
def encode_image(image_path, message, output_path):
    img = Image.open(image_path)
    binary_message = ''.join(format(ord(i), '08b') for i in message)
    data_index = 0
    img_data = img.getdata()
    new_img_data = []

    for pixel in img_data:
        pixel_list = list(pixel)
        for i in range(3):  # for R, G, B channels
            if data_index < len(binary_message):
                pixel_list[i] = (pixel_list[i] & 0xFE) | int(binary_message[data_index])
                data_index += 1
        new_img_data.append(tuple(pixel_list))

    img.putdata(new_img_data)
    img.save(output_path)

# Function to extract message from image
def decode_image(image_path):
    img = Image.open(image_path)
    img_data = img.getdata()
    binary_message = ''

    for pixel in img_data:
        for i in range(3):  # for R, G, B channels
            binary_message += str(pixel[i] & 1)

    # Convert binary to text
    decoded_message = ''
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        decoded_message += chr(int(byte, 2))

    return decoded_message.rstrip('\x00')

# Example usage
encode_image('test2.jpeg', 'Hello, World!', 'output_image.png')
decoded_message = decode_image('output_image.png')
print(decoded_message)


Hello, World!¥mKsñ¶¤üøüÇ$$IþÛRU×OÿÿÿÚ¯ÁµÖªm¶ÛI$´  'þIþHÿÿ         ÿÁ      à?Çÿÿÿ  -VÚÕãóêv«UI*Ç$I$I$I$I$IÿòRI*JÚU¦­$I$I$I$I'ÿÿÿÿÏ$I$I$Iüÿÿò$I$Iüÿÿÿÿ$I$I$I$I?ÿÿÿÿù$I<sÿ À'ÿø$I$IäÿÿÿÁüÿ$òIÿÿÏ'ÿÉÿÿ É?ÿù$I$I$IÿÏçþOàÿÇ	ãþIäqÁüÿÿÿÿ$I$Iüÿäâ?æ×k­·I[RÿþO<þO$I$I$IüÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÉàqÏ?ÿùãþG Á ø?ÿ'ÿÿ8¹$Iÿÿÿ;yÛq$y¶Ûm¶ÛmÇàO?ðÿÿÿÿO$yÿÿíÿòI8IÿòI$ÿÿÿÿçòqäãÉûbNÿÿÿÿÿÿÿÿÿÿûmÿÿÿÿÿÿ¶Ûm¶ÛmþGÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿ'øÿÿÿ$ÿ ñÁÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿI$ÿÿ'}Ï;ãÿàþG?ÿÿãÏ$Iüÿÿÿÿÿÿÿç8à~G$IÿÿÿØqÿþIÿÿÿÿÿÿ$ÿÿû·ÿòI$IÿÿÿÿÿÇÿòIÿA$IüväÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿqÀçyÿÿÿÿÿÿjm¶Û¶Ûmm¶Û$õmm²Im¶Û$wÄÈ$I$I$I<pÿþàà÷Uþ·m$I?ÿÿªÚ¹ÿÿÿª«mhvµÚqÇäO9$Iý[R©§Hÿÿÿ)$«Tÿµ¦­I$I$´  'þIÿþH ù         ø      ãóÏüÿÿÿÿ  -¶ªÕ RU»SªÔø$I$I$I$I$IÿòRM¤ÚVÔíU*ª$I$I$I$I'ÿÿÿÿÏ$I$I$Iüÿÿò$I$Iüÿÿÿÿ$I$I$I$I?ÿÿÿÿù$I?'ð9<H$I$IäÿÀüÿ?ñÿÿÏÿÿ ÿÏ É?ÿù$I$I$IäþàÿçîxäpxàsÉüÿÿÿÿ$I$Iüÿäâ?áZÒÇ]VÚ­~<þO$I$I$Iüÿÿÿÿÿÿÿÿ

In [ ]:
import cv2
import numpy as np

def hide_image(carrier_image_path, secret_image_path, output_image_path):
    # Read the carrier and secret images
    carrier = cv2.imread(carrier_image_path)
    secret = cv2.imread(secret_image_path)

    # Resize the secret image to fit inside the carrier image
    secret_resized = cv2.resize(secret, (carrier.shape[1], carrier.shape[0]))

    # Convert both images to grayscale (optional, but ensures you are hiding in a single channel)
    carrier_gray = cv2.cvtColor(carrier, cv2.COLOR_BGR2GRAY)
    secret_gray = cv2.cvtColor(secret_resized, cv2.COLOR_BGR2GRAY)

    # Normalize the secret image to fit in the least significant bit (LSB)
    secret_binary = secret_gray // 255  # Convert secret image to binary (0 or 1)

    # Embed the secret in the carrier image using the LSB method
    carrier_with_secret = carrier.copy()

    # Use the secret image to overwrite the least significant bit in the carrier image
    carrier_with_secret[..., 0] = (carrier_with_secret[..., 0] & 0xFE) | secret_binary
    carrier_with_secret[..., 1] = (carrier_with_secret[..., 1] & 0xFE) | secret_binary
    carrier_with_secret[..., 2] = (carrier_with_secret[..., 2] & 0xFE) | secret_binary

    # Save the output image
    cv2.imwrite(output_image_path, carrier_with_secret)

    print("Image with secret embedded saved at", output_image_path)

# Usage example
carrier_image_path = '/content/test2.jpeg'
secret_image_path = '/content/test4.png'
output_image_path = '/content/test.png'

hide_image(carrier_image_path, secret_image_path, output_image_path)


Image with secret embedded saved at /content/test.png


In [ ]:
def extract_image_from_carrier(carrier_image_path, output_image_path):
    # Read the carrier image
    carrier = cv2.imread(carrier_image_path)

    # Extract the least significant bit from each channel
    extracted_secret = (carrier[..., 0] & 1) * 255

    # Convert the extracted data back to a usable image
    extracted_secret = np.uint8(extracted_secret)

    # Save the extracted image
    cv2.imwrite(output_image_path, extracted_secret)

    print("Extracted secret image saved at", output_image_path)

# Usage example
extracted_image_path = '/content/test5.png'
extract_image_from_carrier(output_image_path, extracted_image_path)


Extracted secret image saved at /content/test5.png
